# H&M Personalized Fashion Recommendations (4등 기록)
[대회 링크](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/324094)

- review : 해당 캐글러는 혼자 골드 메달을 따고 싶어서 스스로 진행하였음

</br>

## 1. 파이프 라인

![-](https://i.postimg.cc/CMWhdC2L/arch.png)

  1. recall models : 여러 모델들을 통해 추천 후보군을 생성함
    - 사용자 관심 기반으로 추천할 수 있는 후보 아이템들을 뽑아내는 과정

  2. ranking models : 다수의 후보 아이템의 순위를 매겨 상위권 아이템 결정 작업 진행

  3. TOP 12 : 상위권 아이템중 12개의 아이템을 최종 추천 리스트로 선정함

 </br>

**recall models를 이용하여 다수의 추천 후보를 생성 후 ranking model을 통해 상위 아이템들을 선정하여 최종적으로 추천하는 파이프 라인**

 </br>

## 1 - 1. recall models 사용 모델

  1. Two Tower MMoE(멀티모달 확장 모델) : 사용자와 아이템의 특성을 학습하고
 상호작용을 최적화 하는 방식으로 추천을 생성함
  - 사용자와 아이템의 특성을 다른 타워에서 학습하고 이를 결합해 추천해주는 과정
  - 필자는 **Gating Network**를 사용하여 활성 사용자와 비활성 사용자를 구분함
  - 해당 네트워크를 통해 집단에 특성에 맞는 다른 전문가 네트워크를 사용하였음
  - **ex) 활성화 사용자 >> 새로운 상품에 더 흥미가 감**
  - **ex) 비활성 사용자 >> 과거에 구매한 상품을 다시 구매할 가능성이 높음**

 </br>

  2. Item2item CF(협업 필터링) : 사용자 행동 패턴을 기반으로 비슷한 아이템 추천
    - 아이템 간 유사도를 바탕으로 진행함
    - 두 아이템 간의 관계를 기반으로 Y를 구매한 사용자가 Z도 구매했다는 패턴을 이용
    
  </br>

  3. repurchase(재구매 모델) : 사용자가 이전에 구매했던 아이템을 기반하여 반복 구매 가능성이 높은 상품 추천 모델
    - 각 고객이 최근에 구매한 20개의 상품을 기반으로 함

  </br>

  4. Popular(인기 아이템 모델) : 사용자들에게 인기 있는 아이템 추천
    - 많은 사용자가 구매하거나 조회한 아이템을 추천하는 방식
    - 지난 주에 인기 있었던 상품 순위를 기반으로 한 모델

   </br>

**각 모델들이 추천한 아이템을 candidates에 저장함**

 </br>

## 1 - 2. ranking model 사용 모델

  1. LightGBM : Gradient Boosting 방식 결정 알고리즘
    - 빠르고 정확한 예측이 가능함
    - 다양한 특성을 고려해 아이템의 중요도 평가

</br>

  2. DCN : 깊은 신경망과 교차 기능을 활용한 네트워크
    - 특성 간의 상호작용을 학습하여 아이템 순위를 매김

</br>

## 2. 학습 방식
  
  1. Cross Validation : 데이터셋을 여러개의 폴드로 나누어 학습,검증 데이터로 나누어 사용하는 방식
    - 해당 필자는 5개의 폴드를 생성하여 3개의 폴드를 학습데이터로 생성



## 3. 콜드 스타트 문제

  - 콜드 스타트란? : 추천 시스템에서 초기 데이터가 부족할 때 발생하는 문제

  1. 사용자 콜드 스타트 : 새로운 사용자가 회원 등록을 하면 과거 데이터가 없으므로 어떤걸 추천해줘야할지 모름
  
  - 필자는 Two Tower MMoE 모델을 이용하여 데이터가 없는 고객들에게도 사용자 인구 통계 특성을 사용하여 후보 아이템 생성 가능

  </br>

  2. 아이템 콜드 스타트 : 새로운 상품이 추가 되었을 때 해당 상품에 대한 데이터가 없음
  - 필자는 상품의 메타데이터 외에도 텍스트와 이미지에서 특징을 추출하였음


  **최종 로컬 교차 검증 점수 : 0.039 / LB점수 : 0.0349**